In [4]:
!pip install pybluez
#!pip install face_recognition
%pip install opencv-python
%pip install https://pypi.python.org/packages/da/06/bd3e241c4eb0a662914b3b4875fc52dd176a9db0d4a2c915ac2ad8800e9e/dlib-19.7.0-cp36-cp36m-win_amd64.whl#md5=b7330a5b2d46420343fbed5df69e6a3f


  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached dlib-19.24.1.tar.gz (3.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for dlib
Failed to build dlib
  Running setup.py install for dlib: started
  Running setup.py install for dlib: still running...
  Running setup.py install for dlib: still running...
  Running setup.py install for dlib: finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [302 lines of output]
      running bdist_wheel
      running build
      running build_py
      package init file 'tools\python\dlib\__init__.py' not found (or not a regular file)
      running build_ext
      Building extension for Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
      Invoking CMake setup: 'cmake C:\Users\dell\AppData\Local\Temp\pip-install-lsijiisd\dlib_e8ef8c3e41fd4ac29009958977f47488\tools\python -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=C:\Users\dell\AppData\Local\Temp\pip-install-lsijiisd\dlib_e8ef8c3e41fd4ac29009958977f47488\build\lib.win-amd64-3.9 -DPYTHON_EXECUTABLE=d:\software\anaconda3\python.exe -DCMAKE_LIBRARY_OUTPUT_DIRECTORY_RELEASE=C:\Users\dell\AppData\Local\Temp\pip-install-lsijiisd\dlib_e8ef8c3e41fd4ac29009958977f47488\build\lib.win-amd64-3.9 -A x64'
      -- Building for: Visual Studio 16 2019
      -- Selecting

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: dlib-19.7.0-cp36-cp36m-win_amd64.whl is not a supported wheel on this platform.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tkinter as tk
#import face_recognition
import cv2
import numpy as np
import bluetooth
import csv

In [3]:

#bluetooth
def discovery():

    index = 1
    nearby_devices = bluetooth.discover_devices(lookup_names=True)
    #print("found %d devices" % len(nearby_devices))

    # Define the filename for the CSV file
    filename = 'Bluetooth/Known_users.csv'
    with open(filename, mode='r') as csv_file:
        content = csv.reader(filename)
        for addr, name in nearby_devices:
            if nearby_devices[0][0] == addr and nearby_devices[0][1] == name:
                thing = "Welcome "+name
                label.configure(text = thing)
                print(thing)
            else:
                with open(filename, mode='w',newline='') as csv_file:
                    # Create a writer object
                    writer = csv.writer(csv_file)
                    for addr, name in nearby_devices:
                        # Write the data to the CSV file
                        writer.writerow([addr,name])
                        index = index + 1
                        #print(" %s - %s" % (addr, name))
                        #thing = "new user detected"
                        label.configure(text = thing)
                        print(thing)
    

In [4]:
def ohyeah():
    # Get a reference to webcam #0 (the default one)
    video_capture = cv2.VideoCapture(1)

    # Load a sample picture and learn how to recognize it.
    obama_image = face_recognition.load_image_file("people/obama.jpg")
    obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

    # Load a second sample picture and learn how to recognize it.
    zeyad_image = face_recognition.load_image_file("people/zeyad.jpg")
    zeyad_face_encoding = face_recognition.face_encodings(zeyad_image)[0]

    # Load a third sample picture and learn how to recognize it.
    kamala_image = face_recognition.load_image_file("people/kamala.jpg")
    kamala_face_encoding = face_recognition.face_encodings(kamala_image)[0]

    # Load a fourth sample picture and learn how to recognize it.
    ayman_image = face_recognition.load_image_file("people/Dr.ayman ezzat.jpg")
    ayman_face_encoding = face_recognition.face_encodings(ayman_image)[0]

    # Create arrays of known face encodings and their names
    known_face_encodings = [obama_face_encoding,zeyad_face_encoding,kamala_face_encoding,ayman_face_encoding]
    known_face_names = ["Barack Obama","zeyad medhat","kamala harris","Dr. ayman ezzat"]

    # Initialize some variables
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()

        # Only process every other frame of video to save time
        #if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        #small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = frame[:, :, ::-1]

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

        process_this_frame = not process_this_frame

        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            #cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            #cv2.rectangle(frame, (left, bottom - 35),
            #                (right, bottom), (0, 0, 255), cv2.FILLED)
            #font = cv2.FONT_HERSHEY_DUPLEX
            #cv2.putText(frame, name, (left + 6, bottom - 6),font, 1.0, (255, 255, 255), 1)
        # Display the resulting image
    #    cv2.imshow('Video', frame)
        # Hit 'q' on the keyboard to quit!
    #    if cv2.waitKey(1) & 0xFF == ord('q'):
    #        break


    # Release handle to the webcam
    #video_capture.release()
    #cv2.destroyAllWindows()

In [6]:

# Create the main window
root = tk.Tk()

root.after(0, discovery)
# Set the title and size of the window
root.title("ADHD SUPPORT SYSTEM AAAAAAA!!!")
root.geometry("400x300")
# Add a label to the window
label = tk.Label(root, text="Hello, World!")
label.pack()
# Add a button to the window
#bluetooth_button = tk.Button(root, text="Bluetooth discovery",command=discovery)
#bluetooth_button.pack()
# Add a button to the window
button = tk.Button(root, text="Face ID",command = ohyeah)
button.pack()
# Add a button to the window
button = tk.Button(root, text="Click me!")
button.pack()
# Add a button to the window
button = tk.Button(root, text="Click me!")
button.pack()
# Run the main loop
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\software\anaconda3\envs\deeplearning\lib\tkinter\__init__.py", line 1885, in __call__
    return self.func(*args)
  File "d:\software\anaconda3\envs\deeplearning\lib\tkinter\__init__.py", line 806, in callit
    func(*args)
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_16748\58376256.py", line 3, in discovery
    import bluetooth
ModuleNotFoundError: No module named 'bluetooth'
